In [13]:
!wget https://github.com/karpathy/minbpe/blob/master/tests/taylorswift.txt

--2024-07-29 14:16:17--  https://github.com/karpathy/minbpe/blob/master/tests/taylorswift.txt
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘taylorswift.txt.1’

taylorswift.txt.1       [ <=>                ] 998.89K  --.-KB/s    in 0.009s  

2024-07-29 14:16:17 (103 MB/s) - ‘taylorswift.txt.1’ saved [1022861]



In [14]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]): # Pythonic way to iterate consecutive elements
                                          # ids[1:] --> creates a new list that is the original list ids starting from the second element
                                          # zip(ids, ids[1:]) --> pairs each element in ids with its subsequent element, forming a tuple.
                                              # this loop iterates over each of these pairs
        counts[pair] = counts.get(pair, 0) + 1    # returns the valye for the key "pair" from dictionary "counts" // if not already a key, returns 0 instead (default value) // then increments count of "pair" by 1
    return counts

In [15]:
def merge(ids, pair, idx): # Defines a function named merge that takes three parameters:
                                      # ids (a list of integers), pair (a tuple of two integers to search for in ids), and idx (a new integer that will replace occurrences of pair).
  # in the list of ints (ids), replace all consecutive occurences of pair with the new token idx
  newids = []
  i = 0
  while i < len(ids):
    # if we are not at the very last position AND the pair matches, replace it
    if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
      newids.append(idx)
      i += 2
    else:
      newids.append(ids[i])
      i += 1

  return newids

# print(merge([5, 6, 6, 7, 9, 1], (6, 7), 99))

# tokens2 = merge(tokens, top_pair, 256)

In [16]:
# import regex as re
# gpt2pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""") # GPT2's regex

# print(re.findall(gpt2pat, "Hello've world123 how's are you!!!?"))

In [17]:
text = ""
with open('taylorswift.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [18]:
import regex as re

GPT4_SPLIT_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""

gpt4_tokens = re.findall(GPT4_SPLIT_PATTERN, text)
gpt4_tokens = [token.encode("utf-8") for token in gpt4_tokens]
gpt4_tokens = [list(map(int, token)) for token in gpt4_tokens]
print(gpt4_tokens[0:75])

[[10, 10, 10, 10, 10, 10], [60, 33], [68, 79, 67, 84, 89, 80, 69], [32, 104, 116, 109, 108], [62, 10], [60, 104, 116, 109, 108], [10], [32], [32, 108, 97, 110, 103], [61, 34], [101, 110], [34, 10], [32, 32, 10], [32], [32, 100, 97, 116, 97], [45, 99, 111, 108, 111, 114], [45, 109, 111, 100, 101], [61, 34], [97, 117, 116, 111], [34], [32, 100, 97, 116, 97], [45, 108, 105, 103, 104, 116], [45, 116, 104, 101, 109, 101], [61, 34], [108, 105, 103, 104, 116], [34], [32, 100, 97, 116, 97], [45, 100, 97, 114, 107], [45, 116, 104, 101, 109, 101], [61, 34], [100, 97, 114, 107], [34, 10], [32], [32, 100, 97, 116, 97], [45, 97], [49, 49], [121], [45, 97, 110, 105, 109, 97, 116, 101, 100], [45, 105, 109, 97, 103, 101, 115], [61, 34], [115, 121, 115, 116, 101, 109], [34], [32, 100, 97, 116, 97], [45, 97], [49, 49], [121], [45, 108, 105, 110, 107], [45, 117, 110, 100, 101, 114, 108, 105, 110, 101, 115], [61, 34], [116, 114, 117, 101], [34, 10], [32], [32, 62, 10, 10, 10, 10], [32], [32, 60], [104, 10

In [19]:
"""
tokens = text.encode("utf-8") # raw bytes
print(tokens[0:75])
tokens = list(map(int, tokens)) # convert to a list of integers in range 0..255 for convenience // applies int() function
print('---')
print(tokens[0:75])
print("length:", len(tokens))
print("max:"   , max(tokens))
"""

'\ntokens = text.encode("utf-8") # raw bytes\nprint(tokens[0:75])\ntokens = list(map(int, tokens)) # convert to a list of integers in range 0..255 for convenience // applies int() function\nprint(\'---\')\nprint(tokens[0:75])\nprint("length:", len(tokens))\nprint("max:"   , max(tokens))\n'

In [20]:
def get_stats(ids, current_state):
    # counts = {}
    for pair in zip(ids, ids[1:]):
        current_state[pair] = current_state.get(pair, 0) + 1
    return current_state

def merge(comp_ids, pair, idx):
  newids = []
  for i in range(len(comp_ids)):
    newids.append([])
  j = 0
  for ids in comp_ids:
    i = 0
    while i < len(ids):
      if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
        newids[j].append(idx)
        i += 2
      else:
        newids[j].append(ids[i])
        i += 1
    j += 1
  return newids

# ---
vocab_size = 10000 # the target final vocabulary size
num_merges = vocab_size - 256
ids = gpt4_tokens # copy so we don't destroy the original list // preservation for comparison

merges = {} # (int, int) -> int
for i in range(num_merges):
  stats = {}
  for subword in ids:
    stats = get_stats(subword, stats)
  pair = max(stats, key=stats.get) # stats.get fetches value assosciated with each key in stats // this helps to determine key with highest value
  idx = 256 + i
  print(f"merging {pair} into a new token {idx}")
  ids = merge(ids, pair, idx)      # replaces all occurences of "pair" in "ids" with new tokens "idx"
  merges[pair] = idx               # records merge operation in "mergs" dictiionary, mapping from pair of tokens to the new token index

merging (7775, 626) into a new token 7776
merging (7776, 564) into a new token 7777
merging (48, 682) into a new token 7778
merging (425, 3338) into a new token 7779
merging (7779, 499) into a new token 7780
merging (77, 99) into a new token 7781
merging (3899, 4789) into a new token 7782
merging (1551, 953) into a new token 7783
merging (266, 1126) into a new token 7784
merging (7784, 1842) into a new token 7785
merging (3647, 2313) into a new token 7786
merging (3301, 1986) into a new token 7787
merging (3353, 1986) into a new token 7788
merging (1685, 1791) into a new token 7789
merging (1689, 66) into a new token 7790
merging (1559, 3881) into a new token 7791
merging (70, 1285) into a new token 7792
merging (6150, 2107) into a new token 7793
merging (77, 1333) into a new token 7794
merging (2012, 5638) into a new token 7795
merging (7795, 2107) into a new token 7796
merging (1590, 879) into a new token 7797
merging (1559, 83) into a new token 7798
merging (104, 634) into a new tok

In [21]:
vocab = {idx: bytes([idx]) for idx in range(256)} # initislises dictionary where key key is an interger randing from 0 to 255 // and corresponding value is a byte array containing that integer.
                                                        # this essentially maps each single-byte value to its byte representation. Which is useful for decoding operations becuase it directly translates
                                                        # byte values into their respective byte representations.
for (p0, p1), idx in merges.items(): # iterates through merges dict which contains mappings of pairs of newly merged token indices
    vocab[idx] = vocab[p0] + vocab[p1] # idx is correct. Value was correctly assigned as value during for loop at first!

def decode(ids):
  # given ids (list of integers), return Python string
  tokens = b"".join(vocab[idx] for idx in ids) # b"" means byte string // this concatenates all the byte array from vocab into a single byte array

  text = tokens.decode("utf-8", errors="replace") # Convert Bytes to String: tokens.decode("utf-8", errors="replace") converts the byte array into a string using UTF-8 encoding.
                                                          # The errors='replace' parameter ensures that if any bytes cannot be decoded properly, they are replaced with a placeholder character
                                                          # (typically �) instead of raising an error.
  return text

print(decode([128]))

�


In [22]:
def encode(text):
  # given a string, return list of integers (the tokens)
  tokens = list(text.encode("utf-8"))
  while len(tokens) >= 2: # continues process as long as there's at least 2 tokens in the list.
    temp_dict = {}
    stats = get_stats(tokens, temp_dict) # previously defined // calculate the frequency of each consecutive pair of tokens in the list
    pair = min(stats, key=lambda p: merges.get(p, float("inf"))) # purpose of line is to find pair of tokens "p" that is least preferable for merging based on whether are they defined in "merges" dictionary
                                                                 # if p is not found, basically will return infinity as default value, effectively prioritising it as least desirable. Because infinite value will always
                                                                 # be greater than any index.

                                                                # further notes:
                                                                      # refer to above. but i generally get the consensus. Due to how lambda function works here, p is all pair chars in stats being iterated over.
                                                                      # when p is not in merges, infinity value is set making it impossible to be ever chosen
                                                                      # on the other hand, if not, then the value inside merges or the merge pair vocab will be returned. The lowest value will always be chosen first
                                                                      # which due to how most frequent values are always added first, this means that more frequent values are iterated first etc.

                                                                      # REFER TO NOTES ABOVE
    if pair not in merges:
      break # nothing else can be merged // exits loop when
    idx = merges[pair]
    tokens = merge(tokens, pair, idx)
  return tokens

In [23]:
decode(encode("hello world"))

TypeError: object of type 'int' has no len()

In [ ]:
print(vocab)
print({k: v.decode('utf-8', errors="replace") for k, v in vocab.items()})
